## Fitting a line function

In [1]:
%run "data_processing_.ipynb"
from scipy.optimize import leastsq

In [10]:
def fit(func, line):
    xdata = np.linspace(0, len(line) - 1, len(line))
    ydata = line
    popt, pcov = curve_fit(func, xdata, ydata, maxfev = 100000000)
    # popt, pcov = curve_fit(func, xdata, ydata, max_nfev = 100000000)
    return popt

## getting maximum value of a fitting curve

In [6]:
def get_max(line, func):
    par = fit(func, line)
    values = func(np.linspace(0, len(line) - 1, len(line)), *par) 
    return np.max(values)

## get maxs of every x

In [4]:
def get_maxs(data,func):
    maxs = np.zeros(X_right_boundry - X_left_boundry)
    for i in range(X_right_boundry - X_left_boundry):
        line = data[i,:]
        line_max = get_max(line, func)
        maxs[i] = line_max
    return maxs

# this is for testing

# def get_maxs(data,func):
#     maxs = np.zeros(Y_right_boundry - Y_left_boundry)
#     for i in range(Y_right_boundry - Y_left_boundry):
#         line = data[:,i]
#         line_max = get_max(line, func)
#         maxs[i] = line_max
#     return maxs

## get final fitting peak dR / R (one value) of a frame

In [6]:
def peak_dRoverR(file, b_file, func, maxs_func):
    dRoverR = get_dRoverR(file, b_file)
    maxs = get_maxs(dRoverR, func)
    peak = get_max(maxs, maxs_func)
    return peak

## get dR / R of every point of interested in a frame

In [7]:
def get_dRoverR(file, b_file):
    blue = get_data(file)
    b_blue = get_data(b_file)
    dRoverR = (blue - b_blue) / b_blue
    return dRoverR

## fit x,y to z surface

In [3]:
def fit_xy_to_z_surface(x, y, z, func, par):
    error_func = lambda p: np.ravel(func(*p)(x, y) - z)
    pfit, pcov, infodict, errmsg, success = leastsq(error_func, par, full_output=1)
    return pfit, pcov, infodict